In [1]:
from tensorflow import keras


2022-07-14 03:04:06.332422: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: :/opt/gurobi912/linux64/lib
2022-07-14 03:04:06.332441: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


In [5]:
def f1(y_true, y_pred):
    return 1
def precision(y_true, y_pred):
    return 1
def recall(y_true, y_pred):
    return 1

def coeff_determination(y_true, y_pred):
  SS_res =  K.sum(K.square( y_true-y_pred ))
  SS_tot = K.sum(K.square( y_true - K.mean(y_true) ) )
  return ( 1 - SS_res/(SS_tot + K.epsilon()) )

metrics = {
    'f1': f1,
    'precision': precision,
    'recall': recall,
    'coeff_determination': coeff_determination,
}
model = keras.models.load_model('ignored/vgg_250_epochs/', custom_objects=metrics)

In [11]:
from pathlib import Path
image_path = Path('../../data/ignored/images/imageLD/00000064/1.jpg')

In [ ]:
from keras.preprocessing.image import ImageDataGenerator
datagen = ImageDataGenerator(rescale=1/255., )

In [ ]:
import pandas as pd

In [ ]:
img_data = pd.DataFrame({})

In [ ]:
img_generator = datagen.flow_from_dataframe(dataframe=data_teste, directory=path_img,
                                             target_size=(img_height, img_width),
                                             x_col='path',
                                             y_col=data_treino.columns[:-1],
                                             class_mode="raw",
                                             shuffle=False,
                                             seed=7)

In [10]:
x.shape

(224, 224, 3)

In [9]:
model.fit(x)

ValueError: in user code:

    File "/home/vinicius/academia/extensao/mdc-20221-trabalho_final/venv/lib/python3.8/site-packages/keras/engine/training.py", line 1051, in train_function  *
        return step_function(self, iterator)
    File "/home/vinicius/academia/extensao/mdc-20221-trabalho_final/venv/lib/python3.8/site-packages/keras/engine/training.py", line 1040, in step_function  **
        outputs = model.distribute_strategy.run(run_step, args=(data,))
    File "/home/vinicius/academia/extensao/mdc-20221-trabalho_final/venv/lib/python3.8/site-packages/keras/engine/training.py", line 1030, in run_step  **
        outputs = model.train_step(data)
    File "/home/vinicius/academia/extensao/mdc-20221-trabalho_final/venv/lib/python3.8/site-packages/keras/engine/training.py", line 889, in train_step
        y_pred = self(x, training=True)
    File "/home/vinicius/academia/extensao/mdc-20221-trabalho_final/venv/lib/python3.8/site-packages/keras/utils/traceback_utils.py", line 67, in er

In [ ]:
import tensorflow as tf
from keras import layers
from keras.applications import VGG16
from keras.models import Model
pre_trained_model  = VGG16(include_top=False, input_shape=(img_height,img_width,3), weights='imagenet')
transfer_layer = pre_trained_model.get_layer('block5_pool')
conv_model = Model(inputs=pre_trained_model.input, outputs=transfer_layer.output)

In [ ]:
from keras.models import Sequential
from keras.layers import Dense, Flatten, Dropout, BatchNormalization
# Start a new Keras Sequential model.
model = Sequential()

# Add the convolutional part of the VGG16 model from above.
model.add(conv_model)
model.add(BatchNormalization())
# Flatten the output of the VGG16 model because it is from a
# convolutional layer.
model.add(Flatten())

# Add a dense (aka. fully-connected) layer.
# This is for combining features that the VGG16 model has
# recognized in the image.
model.add(Dense(1024, activation='relu'))

# Add a dropout-layer which may prevent overfitting and
# improve generalization ability to unseen data e.g. the test-set.
model.add(Dropout(0.1))

# Add the final layer for the actual classification.
labels = 40
model.add(Dense(labels, activation='sigmoid'))

In [ ]:
model.summary()

In [ ]:
from keras.utils.vis_utils import plot_model
plot_model(model, show_shapes = True)

In [ ]:
input_shape = pre_trained_model.layers[0].output_shape[0][1:3]
input_shape

In [ ]:
from keras import backend as K

def coeff_determination(y_true, y_pred):
  SS_res =  K.sum(K.square( y_true-y_pred ))
  SS_tot = K.sum(K.square( y_true - K.mean(y_true) ) )
  return ( 1 - SS_res/(SS_tot + K.epsilon()) )

In [ ]:
from keras.optimizers import Adam
optimizer = Adam(learning_rate=0.0001, beta_1=0.9, beta_2=0.999, epsilon=1e-3)
model.compile(optimizer = optimizer, loss='mean_absolute_error', metrics=[coeff_determination])

In [ ]:
model.load_weights('weights_250_byron.h5')

In [ ]:
def lr_schedule(epoch):
    lrate = 0.001
    if epoch > 75:
        lrate = 0.0005
    elif epoch > 100:
        lrate = 0.0003
    return lrate

In [ ]:
from keras import utils, callbacks
from keras.callbacks import LearningRateScheduler
earlystopping = callbacks.EarlyStopping(monitor='val_loss', mode='min', patience= 15, restore_best_weights=True)
epochs = 30

